In [1]:
from symred.symplectic import ObservableGraph

In [66]:
from symred.chem import MoleculeBuilder
from symred.projection import QubitTapering, CS_VQE
from symred.symplectic import QuantumState, find_symmetry_basis
import numpy as np
import json

In [67]:
with open('data/molecule_data.json', 'r') as jfile:
    molecule_geometries = json.load(jfile)

In [68]:
# Set molecule parameters
speciesname = 'Be_STO-3G_SINGLET'
#print(speciesname in scoring_data)
mol_data = molecule_geometries[speciesname]
if 'name' in mol_data:
    print(mol_data['name'])
    
atoms  = mol_data['atoms']
coords = mol_data['coords']
basis  = mol_data['basis'] #'6-31g' #'cc-pVDZ' #
spin   = mol_data['multiplicity']
charge = mol_data['charge']
geometry = list(zip(atoms, coords))
molecule = MoleculeBuilder(geometry=geometry, charge=charge, basis=basis)

Molecule geometry:
Be	0	0	0

HF converged?   True
CCSD converged? True
FCI converged?  True

HF energy:   -14.351880476202023
MP2 energy:  -14.376238850841515
CCSD energy: -14.403650751752787
FCI energy:  -14.403655108067682


Number of qubits: 10


In [69]:
taper_hamiltonian = QubitTapering(molecule.H_q)

print(f'We are able to taper {taper_hamiltonian.n_taper} qubits from the Hamiltonian.\n')
print('The symmetry generators are\n')
print(taper_hamiltonian.symmetry_generators)
print('\nand may be rotated via a sequence of Clifford pi/2 rotations\n')
print(taper_hamiltonian.stabilizers.stabilizer_rotations)
print('\nonto the single-qubit Pauli operators\n')
print(taper_hamiltonian.stabilizers.rotate_onto_single_qubit_paulis())
hf_array = molecule.H_fermion.hf_comp_basis_state
taper_hamiltonian.stabilizers.update_sector(hf_array)
print(f'\nThe symetry sector corresponding with the reference {hf_array} is {taper_hamiltonian.stabilizers.coeff_vec}')
ham_tap = taper_hamiltonian.taper_it(ref_state=hf_array)
#sor_tap = taper_hamiltonian.taper_it(aux_operator=sor_ham, ref_state=hf_array)
#sor_tap.coeff_vec/=np.linalg.norm(sor_tap.coeff_vec)
ucc_tap = taper_hamiltonian.taper_it(aux_operator=molecule.T_q, ref_state=hf_array)
n_taper = taper_hamiltonian.n_taper
tapered_qubits   = taper_hamiltonian.stab_qubit_indices
untapered_qubits = taper_hamiltonian.free_qubit_indices
hf_tapered = taper_hamiltonian.tapered_ref_state

print(f'The tapered Hartree-Fock state is', QuantumState([hf_tapered]))

We are able to taper 5 qubits from the Hamiltonian.

The symmetry generators are

 1 ZIZIIZIZIZ 
 1 IZIZIZIZIZ 
 1 IIIIZZIIII 
 1 IIIIIIZZII 
 1 IIIIIIIIZZ

and may be rotated via a sequence of Clifford pi/2 rotations

None

onto the single-qubit Pauli operators

-1 XIIIIIIIII 
-1 IXIIIIIIII 
-1 IIIIXIIIII 
-1 IIIIIIXIII 
-1 IIIIIIIIXI

The symetry sector corresponding with the reference [1 1 1 1 0 0 0 0 0 0] is [1 1 1 1 1]
The tapered Hartree-Fock state is  1.0000000000 |11000>


In [70]:
cs_vqe = CS_VQE(ham_tap, hf_tapered, basis_weighting_operator=ucc_tap)

In [71]:
print(cs_vqe.decomposed)

{'symmetry': -8.672+0.000j IIIII +
-0.047+0.000j IZIII +
-0.297+0.000j IIZII +
0.183+0.000j IZZII +
-0.297+0.000j IIIZI +
0.183+0.000j IZIZI +
0.389+0.000j IIZZI +
0.281+0.000j IZZZI +
-0.297+0.000j IIIIZ +
0.183+0.000j IZIIZ +
0.389+0.000j IIZIZ +
0.281+0.000j IZZIZ +
0.389+0.000j IIIZZ +
0.281+0.000j IZIZZ +
0.263+0.000j IIZZZ +
2.243-0.000j IZZZZ, 'clique_0': -0.047+0.000j ZIIII +
0.670+0.000j ZZIII +
0.183+0.000j ZIZII +
0.183+0.000j ZIIZI +
0.281+0.000j ZIZZI +
0.183+0.000j ZIIIZ +
0.281+0.000j ZIZIZ +
0.281+0.000j ZIIZZ +
2.243-0.000j ZIZZZ +
0.286+0.000j ZZZZZ, 'clique_1': -0.060+0.000j XIIII +
0.038-0.000j XZIII +
0.010+0.000j XZZII +
0.010+0.000j XZIZI +
-0.010+0.000j XZZZI +
0.010+0.000j XZIIZ +
-0.010+0.000j XZZIZ +
-0.010+0.000j XZIZZ +
0.060+0.000j XIZZZ +
-0.038+0.000j XZZZZ}


In [72]:
H_graph = ObservableGraph(ham_tap.symp_matrix, ham_tap.coeff_vec)

In [73]:
C_cliques = H_graph.clique_cover(clique_relation='QWC', colouring_strategy='largest_first')
C0 = C_cliques[0]

print(C0)

0.022+0.000j YYZZZ +
-0.022+0.000j YYIII +
0.263+0.000j IIZZZ +
0.389+0.000j IIZZI +
0.389+0.000j IIZIZ +
0.389+0.000j IIIZZ +
-0.297+0.000j IIZII +
-0.297+0.000j IIIZI +
-0.297+0.000j IIIIZ +
-8.672+0.000j IIIII


In [74]:
from symred.symplectic import PauliwordOp

mask_diagonal = np.where(~np.any(ham_tap.X_block, axis=1))[0]
diag_op = PauliwordOp(ham_tap.symp_matrix[mask_diagonal], ham_tap.coeff_vec[mask_diagonal])

diag_op

-8.672+0.000j IIIII +
-0.047+0.000j ZIIII +
-0.047+0.000j IZIII +
0.670+0.000j ZZIII +
-0.297+0.000j IIZII +
0.183+0.000j ZIZII +
0.183+0.000j IZZII +
-0.297+0.000j IIIZI +
0.183+0.000j ZIIZI +
0.183+0.000j IZIZI +
0.389+0.000j IIZZI +
0.281+0.000j ZIZZI +
0.281+0.000j IZZZI +
-0.297+0.000j IIIIZ +
0.183+0.000j ZIIIZ +
0.183+0.000j IZIIZ +
0.389+0.000j IIZIZ +
0.281+0.000j ZIZIZ +
0.281+0.000j IZZIZ +
0.389+0.000j IIIZZ +
0.281+0.000j ZIIZZ +
0.281+0.000j IZIZZ +
0.263+0.000j IIZZZ +
2.243-0.000j ZIZZZ +
2.243-0.000j IZZZZ +
0.286+0.000j ZZZZZ

In [86]:
C_cliques[0]

0.022+0.000j YYZZZ +
-0.022+0.000j YYIII +
0.263+0.000j IIZZZ +
0.389+0.000j IIZZI +
0.389+0.000j IIZIZ +
0.389+0.000j IIIZZ +
-0.297+0.000j IIZII +
-0.297+0.000j IIIZI +
-0.297+0.000j IIIIZ +
-8.672+0.000j IIIII

In [78]:
find_symmetry_basis(C_cliques[0])

 1 XZIII 
 1 IIZII 
 1 IIIZI 
 1 IIIIZ

In [92]:
from symred.utils import gf2_gaus_elim, gf2_basis_for_gf2_rref
from symred.symplectic import StabilizerOp

operator = C_cliques[0]

ZX_symp = np.hstack([operator.Z_block, operator.X_block])
print(ZX_symp)
reduced = gf2_gaus_elim(ZX_symp)
print(reduced)
kernel  = gf2_basis_for_gf2_rref(reduced)
print(kernel)
stabilizers = ObservableGraph(kernel, np.ones(kernel.shape[0]))
#if np.any(~stabilizers.adjacency_matrix):
#    # if any of the stabilizers are not mutually commuting, take the largest commuting subset
#    stabilizers = stabilizers.clique_cover(clique_relation='C', colouring_strategy='largest_first')[0]

StabilizerOp(stabilizers.symp_matrix, np.ones(stabilizers.n_terms))

[[1 1 1 1 1 1 1 0 0 0]
 [1 1 0 0 0 1 1 0 0 0]
 [0 0 1 1 1 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 0]
 [0 0 1 0 1 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
[[1 1 0 0 0 1 1 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
[[1 0 0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 1]]


 1 XZIII 
 1 IYIII 
 1 ZZIII 
 1 IIZII 
 1 IIIZI 
 1 IIIIZ

In [101]:
gf2_matrix_in_rreform = reduced
print(reduced)
rows_to_columns = gf2_matrix_in_rreform.T
print(rows_to_columns)
eye = np.eye(gf2_matrix_in_rreform.shape[1], dtype=int)
print(eye.T)
# do column reduced form as row reduced form
rrf = gf2_gaus_elim(np.hstack((rows_to_columns, eye.T)))
print(rrf)

zero_rrf = np.where(~rrf[:, :gf2_matrix_in_rreform.shape[0]].any(axis=1))[0]
print(zero_rrf)
basis = rrf[zero_rrf, gf2_matrix_in_rreform.shape[0]:]
print(gf2_matrix_in_rreform.shape[0])
print(basis)

[[1 1 0 0 0 1 1 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
[[1 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
[[1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 1]]
[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0]
 [0 0 

In [77]:
test_op = diag_op + C_cliques[0] + C_cliques[4] + C_cliques[7]
test_op.is_noncontextual

False

In [30]:
import numpy as np
order_by_anticom = np.argsort(np.count_nonzero(~C0.commutes_termwise(C1), axis=1))[::-1]
print(np.where(C0.commutes_termwise(C1)[order_by_anticom[0]]))

(array([3, 7, 9]),)


In [18]:
from symred.symplectic import find_symmetry_basis

sym_0 = find_symmetry_basis(C0)
sym_1 = find_symmetry_basis(C1)

print(sym_0)
print()
print(sym_1)

 1 XIIIX 
 1 IXIXX 
 1 ZIIZZ 
 1 IZIZI 
 1 IIZII

 1 XZZZI 
 1 ZXZZI 
 1 ZZXII 
 1 ZZIXI 
 1 IIIIX


In [19]:
sym_0.commutes_termwise(sym_1)

array([[ True, False, False, False,  True],
       [ True, False, False, False,  True],
       [False,  True,  True, False, False],
       [ True, False,  True, False,  True],
       [ True,  True, False,  True,  True]])

In [14]:
C_union = C0+C1
C_graph = ObservableGraph(C_union.symp_matrix, C_union.coeff_vec)
print(C0)
print()
print(C1)

AC_cliques = C_graph.clique_cover(clique_relation='AC', colouring_strategy='random_sequential')

0.183+0.000j IZIZI +
-0.022+0.000j XYIYI +
2.243-0.000j ZIZZZ +
-0.005+0.000j XIIIX +
0.281+0.000j IZZZI +
0.005-0.000j XZZZX +
-8.672+0.000j IIIII +
-0.297+0.000j IIZII +
0.281+0.000j ZIIZZ +
-0.022+0.000j XXIXI

0.012+0.000j IIXXI +
-0.004+0.000j ZZIXI +
-0.005+0.000j ZXZZX +
0.022-0.000j YYIIX +
-0.022+0.000j YYIII +
-0.004+0.000j IIIIX +
-0.022+0.000j XXXII +
-0.004+0.000j ZZXII +
-0.010+0.000j XZZZI +
0.010+0.000j ZXZZI


In [46]:
print(AC_cliques[0])

0.005+0.000j IYIIY +
-0.010+0.000j ZXZII


In [22]:
import numpy as np
from symred.symplectic import StabilizerOp, find_symmetry_basis

random_Z_block = np.random.randint(0,2,(3, 5))
S = StabilizerOp(np.hstack([np.zeros_like(random_Z_block), random_Z_block]), np.ones(3))
print(S)

1 ZIIZZ 
1 ZZZZZ 
1 IIIZZ


In [23]:
S.update_sector(hf_tapered)

In [24]:
print(S)

-1 ZIIZZ 
1 ZZZZZ 
1 IIIZZ


In [25]:
S.target_sqp = 'X'
print(S.rotate_onto_single_qubit_paulis())

-1 XIIII 
-1 IXIII 
-1 IIIXI


In [26]:
S.target_sqp = 'X'
print(S.rotate_onto_single_qubit_paulis())

-1 XIIII 
-1 IXIII 
-1 IIIXI
